In [248]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(threshold=100000, precision = 2)
e_n = 1.0 # energy
t_n = 1.0 # amplitude for hopping (|r| = 1)


N = 2 # length of cume

def gen_triples(N):
    i, j, k = (0, 0, 0)
    coords = []
    for i in range(N):
        for j in range(N):
            for k in range(N):
                coords.append((i,j,k))

    return coords


tic = time.time()   

coords = gen_triples(N)
mapping = {i: coords[i] for i in range(len(coords))}
inv_mapping = {coords[i]: i for i in range(len(coords))}
print(inv_mapping[(0,1,1)])
# print (coords)
toc = time.time()
print (toc-tic)


sites = N**3
H = np.zeros((sites, sites), dtype = float)
for i in range(sites):
    x, y, z = mapping[i]
    H[i, i] = e_n
    if (z + 1 < N):
        H[i, i+1] = t_n
    if y + 1 < N:
        H[i, i+N] = t_n
    if x + 1 < N:
        H[i, i+N**2] = t_n
    if z > 0:
        H[i, i-1] = t_n
    if y > 0:
        H[i, i-N] = t_n
    if x > 0:
        H[i, i-N**2] = t_n

print (mapping)

print (H)
print (np.allclose(H, H.T, atol=1e-8))

3
0.00020599365234375
{0: (0, 0, 0), 1: (0, 0, 1), 2: (0, 1, 0), 3: (0, 1, 1), 4: (1, 0, 0), 5: (1, 0, 1), 6: (1, 1, 0), 7: (1, 1, 1)}
[[1. 1. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 1. 0. 1. 0. 0.]
 [1. 0. 1. 1. 0. 0. 1. 0.]
 [0. 1. 1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1. 1. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1. 0. 1. 1.]
 [0. 0. 0. 1. 0. 1. 1. 1.]]
True


In [249]:
w, v = np.linalg.eigh(H)
print (v)
print(w)
print (np.linalg.norm(v[:,0]))
print (H.dot(v[:,0]))
print (H.dot(v[:,-1]))
for i in range(8):
    for j in range(8):
        print (i, j, v[:,i].dot(v[:,j]))

[[-3.54e-01  5.59e-01  2.49e-01  4.33e-03 -1.42e-17 -5.91e-01  1.61e-01
  -3.54e-01]
 [ 3.54e-01 -3.19e-01  2.07e-01  4.80e-01  3.17e-01 -3.24e-01 -4.12e-01
  -3.54e-01]
 [ 3.54e-01  4.75e-02 -6.10e-01  3.39e-02  2.60e-01 -6.15e-02  5.51e-01
  -3.54e-01]
 [-3.54e-01 -2.88e-01  1.54e-01 -5.18e-01  5.76e-01  2.06e-01 -2.20e-02
  -3.54e-01]
 [ 3.54e-01 -2.88e-01  1.54e-01 -5.18e-01 -5.76e-01 -2.06e-01  2.20e-02
  -3.54e-01]
 [-3.54e-01  4.75e-02 -6.10e-01  3.39e-02 -2.60e-01  6.15e-02 -5.51e-01
  -3.54e-01]
 [-3.54e-01 -3.19e-01  2.07e-01  4.80e-01 -3.17e-01  3.24e-01  4.12e-01
  -3.54e-01]
 [ 3.54e-01  5.59e-01  2.49e-01  4.33e-03  0.00e+00  5.91e-01 -1.61e-01
  -3.54e-01]]
[-2.00e+00 -1.15e-15 -4.21e-16  5.17e-16  2.00e+00  2.00e+00  2.00e+00
  4.00e+00]
1.0000000000000002
[ 0.71 -0.71 -0.71  0.71 -0.71  0.71  0.71 -0.71]
[-1.41 -1.41 -1.41 -1.41 -1.41 -1.41 -1.41 -1.41]
0 0 1.0000000000000004
0 1 -5.551115123125783e-17
0 2 -2.220446049250313e-16
0 3 -2.7755575615628914e-17
0 4 5.551115

In [250]:
array = []
array1 = []

boundary_condition = {}
z = 1 + 0.1j # energy value
G = np.zeros((sites, sites), dtype = complex)
for triple1 in inv_mapping.keys():
    for triple2 in inv_mapping.keys():
        coord_ = inv_mapping[triple2]
        coord = inv_mapping[triple1]
        G[coord, coord_] = np.sum((v[coord]) * np.conj(v[coord_]) / (z - w))
    
        boundary_condition[triple2] = [np.sum((v[0]) * np.conj(v[coord_]) / (z - w)), 
                        np.sum((v[-1]) * np.conj(v[coord_]) / (z - w))]

import pandas as pd

print (G)
# for x in np.linspace(-20, 20, 100):
#     z = x + 0.1j # energy value
    
#     coord_ = inv_mapping[(0,1,1)]
#     coord = inv_mapping[(0,1,1)]
#     G = np.sum((v[coord]) * np.conj(v[coord_]) / (z - w))
#     boundary_condition[z] = [np.sum((v[0]) * np.conj(v[coord_]) / (z - w)), 
#                             np.sum((v[-1]) * np.conj(v[coord_]) / (z - w))]
#     array.append(G)
#     array1.append(np.sum(np.conj(v[coord]) * (v[coord_]) / (z - w)))
    
#     print (z, boundary_condition[z])
    
# plt.scatter(np.linspace(-20, 20, 100), np.real(array), marker='o')
# plt.scatter(np.linspace(-20, 20, 100), np.real(array1), marker='+')
# plt.show()
# plt.scatter(np.linspace(-20, 20, 100), np.imag(array), marker='o')
# plt.scatter(np.linspace(-20, 20, 100), np.imag(array1), marker='+')
# plt.show()

[[ 5.55e-17-7.70e-02j -3.31e-01-4.53e-17j -3.31e-01+1.82e-17j
   6.94e-18+2.20e-02j -3.31e-01-2.17e-17j -4.16e-16+2.20e-02j
   1.53e-16+2.20e-02j  6.59e-01+6.25e-17j]
 [-3.31e-01-4.53e-17j  1.39e-17-7.70e-02j -5.55e-17+2.20e-02j
  -3.31e-01+3.82e-17j  0.00e+00+2.20e-02j -3.31e-01+6.94e-18j
   6.59e-01-2.60e-17j  1.60e-16+2.20e-02j]
 [-3.31e-01+1.82e-17j -5.55e-17+2.20e-02j  3.16e-16-7.70e-02j
  -3.31e-01-3.47e-18j -1.39e-16+2.20e-02j  6.59e-01-3.47e-17j
  -3.31e-01+3.47e-17j  9.71e-17+2.20e-02j]
 [ 6.94e-18+2.20e-02j -3.31e-01+3.82e-17j -3.31e-01-3.47e-18j
   5.55e-17-7.70e-02j  6.59e-01-1.73e-17j  1.39e-16+2.20e-02j
   0.00e+00+2.20e-02j -3.31e-01-2.69e-17j]
 [-3.31e-01-2.17e-17j  0.00e+00+2.20e-02j -1.39e-16+2.20e-02j
   6.59e-01-1.73e-17j  1.67e-16-7.70e-02j -3.31e-01-1.73e-18j
  -3.31e-01+2.08e-17j  1.60e-16+2.20e-02j]
 [-4.16e-16+2.20e-02j -3.31e-01+6.94e-18j  6.59e-01-3.47e-17j
   1.39e-16+2.20e-02j -3.31e-01-1.73e-18j  1.77e-16-7.70e-02j
  -1.39e-16+2.20e-02j -3.31e-01+1.39e-17j

In [247]:

def num_entries(k, N):
    if 0 <= k <= N - 1:
        return int(1./2 * (k+1) * (k+2))
    elif N <= k <= 2*N - 3:
        return int(1./2 * (-2*k**2 + 6*k*N - 6*k - 3*N**2 +9*N - 4))
    elif 2*N - 2 <= k <= 3*N - 3:
        return int(1./2 * (3*N - 2 - k) * (3*N - 1 - k))
    else:
        return 0



def generate_triples(N, total):
    # List containing generated triples
    gen = []

    # Generate first element
    if 0 <= total <= N - 1:
        i, j, k = (0, 0, total)
        while i <= total:
            # print ("now i is", i)
            while j <= total - i:
                gen.append ((i, j, k))
                j += 1
                k -= 1
            i += 1
            j = 0
            k = total - i
            
                # pass

    elif N <= total <= 2*N - 3:
        i, j, k = (0, total-N+1, N-1)
        while i <= N-1:
            while j <= min(N-1, total - i):
                gen.append ((i, j, k))
                j += 1
                k -= 1
            i += 1
            k = min(N-1, total - i)
            j = total - i - k


            
    elif 2*N - 2 <= total <= 3*N - 3:
        i, j, k = (total - 2*N + 2, N-1, N-1)
        while i <= N-1:
            while j <= min(N-1, total - i):
                gen.append ((i, j, k))
                j += 1
                k -= 1
            i += 1
            k = min(N-1, total - i)
            j = total - i - k
        
    else:
        raise ValueError("Not applicable k")

    return gen

def index_triple_maps(N):
    mapping = {}
    inv_mapping = {}
    for k in range(3*N - 2):
        triples = generate_triples(N, k)
        mapping[k] = {i: triples[i] for i in range(len(triples))}
        inv_mapping[k] = {triples[i]: i for i in range(len(triples))}
    return mapping, inv_mapping

N = 2
mapping, inv_mapping = index_triple_maps(N)
print (mapping)


def calculate(z_n, t_n = 1.0, e_n = 1.0, N = 20, c = complex,
                          boundary_condition = boundary_condition,
                           mapping = mapping, inv_mapping = inv_mapping):
    amplitude = t_n / (z_n - e_n)

    a = {}
    b = {}


    for k in range(3*N-2):
        alpha_num_entries = num_entries(k - 1, N)
        my_entries = num_entries(k, N)
        beta_num_entries = num_entries(k + 1, N)
        a_k = np.zeros((my_entries, alpha_num_entries), dtype=c)
        b_k = np.zeros((my_entries, beta_num_entries), dtype=c)
        for index, triple in mapping[k].items():
            x, y, z = triple

            # Calculate b_k
            if k < 3*N - 3:
                if x < N - 1:
                    b_k[index, inv_mapping[k + 1][x + 1, y, z]] = amplitude

                if y < N - 1:
                    b_k[index, inv_mapping[k + 1][x, y + 1, z]] = amplitude

                if z < N - 1:
                    b_k[index, inv_mapping[k + 1][x, y, z + 1]] = amplitude

            # Calculate a_k
            if 0 < k:
                if x > 0:
                    a_k[index, inv_mapping[k - 1][x - 1, y, z]] = amplitude

                if y > 0:
                    a_k[index, inv_mapping[k - 1][x, y - 1, z]] = amplitude

                if z > 0:
                    a_k[index, inv_mapping[k - 1][x, y, z - 1]] = amplitude

        a[k] = a_k
        b[k] = b_k
    print (a)
    coords = gen_triples(N)
    for x_, y_, z_ in coords:
        sum_ = sum([x_, y_, z_])
        C = np.zeros(shape = (num_entries(sum_, N), 1), dtype = complex)
        C[inv_mapping[sum_][(x_, y_, z_)]] = 1. / (z_n - e_n)
        print ("xyz", x_, y_, z)
        print ("C", C)
#         raise ValueError

        # Calculate Multiplicative Factors 
        A = {}
        G = {}
        if sum_ > 0:
            A[1] = b[1]
            G[1] = a[1] .dot(boundary_condition[(x_, y_, z_)][0])


            for k in range(2, sum_):
                A[k] = np.linalg.solve(np.eye(num_entries(k, N)) - a[k].dot(A[k - 1]), b[k])
                G[k] = np.linalg.solve(np.eye(num_entries(k, N)) - a[k].dot(A[k - 1]), (a[k].dot(G[k-1]))) 

        A[3*N - 4] = a[3*N - 4]
        G[3*N - 4] = b[3*N - 4] .dot(boundary_condition[(x_, y_, z_)][1])
        
        for k in range(3*N - 5, sum_, -1):
            print ("yo", list(range(3*N - 5, sum_, -1)))
            A[k] = np.linalg.solve(np.eye(num_entries(k, N)) - b[k].dot(A[k + 1]), a[k])
            G[k] = np.linalg.solve(np.eye(num_entries(k, N)) - b[k].dot(A[k + 1]), (b[k].dot(G[k+1]))) 

        print (G)
        print (A)
        V = {}
        k = sum_
        if 0 < k < 3*N - 3:
            V[k] = np.linalg.solve(np.eye(num_entries(k, N)) - a[k].dot(A[k - 1]) - b[k].dot(A[k + 1]), 
                                   a[k].dot(G[k-1]) + b[k].dot(G[k+1]) + C)
        if k == 0:
            print ('here')
            V[0] = boundary_condition[(x_, y_, z_)][0]
            V[3*N - 3] = boundary_condition[(x_, y_, z_)][1]
            print (a[1])
            print (b[1])
            V[1] = np.linalg.solve(np.eye(num_entries(1, N)) - b[1].dot(a[2]), 
                                   a[1].dot(V[0]) + (b[1].dot(b[2])).dot(V[3]))
            V[2] = a[2].dot(V[1]) + b[2].dot(V[3])
#             for k in range(1, 3*N - 3):
#                 V[k] = G[k] + A[k].dot(V[k - 1])
        
#             V[k] = np.linalg.solve(np.eye(num_entries(k, N)) - b[k].dot(A[k + 1]), 
#                                     b[k].dot(G[k+1]) + C)
        
        elif k == 3*N - 3:
            V[k] = np.linalg.solve(np.eye(num_entries(k, N)) - a[k].dot(A[k - 1]), 
                                   a[k].dot(G[k-1]) + C)
        for k in range(3*N - 3):
            print ("V",k, V[k])
        break

# #     print (mapping[k])
# #     print (mapping[k-1])
# #     print (a[k])
#     return (V[sum_][inv_mapping[sum_][(4, 2, 1)]]) 

# array1 = []
# for x in np.linspace(-20, 20, 100):
#     z_n = x + 0.1j
asdf = calculate(z_n = 1+0.1j, N = 2)
#     array1.append (asdf)
    

# plt.figure(figsize=(15,10))
# plt.scatter(np.linspace(-20, 20, 100), np.real(array),marker='o')    
# plt.scatter(np.linspace(-20, 20, 100), np.real(array1),marker='+')


{0: {0: (0, 0, 0)}, 1: {0: (0, 0, 1), 1: (0, 1, 0), 2: (1, 0, 0)}, 2: {0: (0, 1, 1), 1: (1, 0, 1), 2: (1, 1, 0)}, 3: {0: (1, 1, 1)}}
{0: array([], shape=(1, 0), dtype=complex128), 1: array([[0.-10.j],
       [0.-10.j],
       [0.-10.j]]), 2: array([[0.-10.j, 0.-10.j, 0. +0.j],
       [0.-10.j, 0. +0.j, 0.-10.j],
       [0. +0.j, 0.-10.j, 0.-10.j]]), 3: array([[0.-10.j, 0.-10.j, 0.-10.j]])}
xyz 0 0 1
C [[0.-10.j]]
yo [1]
{2: array([[6.25e-16-6.59j],
       [6.25e-16-6.59j],
       [6.25e-16-6.59j]]), 1: array([[-0.33-3.11e-17j],
       [-0.33-3.11e-17j],
       [-0.33-3.11e-17j]])}
{2: array([[0.-10.j, 0.-10.j, 0. +0.j],
       [0.-10.j, 0. +0.j, 0.-10.j],
       [0. +0.j, 0.-10.j, 0.-10.j]]), 1: array([[0.-0.02j],
       [0.-0.02j],
       [0.-0.02j]])}
here
[[0.-10.j]
 [0.-10.j]
 [0.-10.j]]
[[0.-10.j 0.-10.j 0. +0.j]
 [0.-10.j 0. +0.j 0.-10.j]
 [0. +0.j 0.-10.j 0.-10.j]]
V 0 (5.551115123125783e-17-0.07703212052614797j)
V 1 [[-0.33-3.25e-17j]
 [-0.33-3.25e-17j]
 [-0.33-3.25e-17j]]
V 2 

In [281]:
v = np.random.uniform(size=(10, 10))
print (v)
print ((np.hstack(np.split(v.reshape(-1), 4))).reshape(10, 10))
# np.split(np.arange(100), 4)

[[0.28 0.84 0.78 0.44 0.87 0.8  0.51 0.19 0.96 0.15]
 [0.   0.57 0.27 0.21 0.81 0.79 0.11 0.22 0.71 0.74]
 [0.71 0.05 0.38 0.34 0.84 0.87 0.52 0.97 0.38 0.09]
 [0.31 0.51 0.29 0.71 0.55 0.49 0.57 0.77 0.63 0.4 ]
 [0.45 0.53 0.48 0.97 0.64 0.99 0.17 0.21 0.52 0.01]
 [0.88 0.79 0.11 0.48 0.04 0.97 0.66 0.32 0.49 0.64]
 [0.33 0.23 0.61 0.39 0.38 0.95 1.   0.48 0.76 0.03]
 [0.75 0.23 0.55 0.63 0.11 0.1  0.82 0.39 0.68 0.41]
 [0.38 0.51 0.71 0.75 0.04 0.5  0.57 0.78 0.47 0.89]
 [0.67 0.08 0.5  0.2  0.73 0.31 0.63 0.18 0.02 0.1 ]]
[[0.28 0.84 0.78 0.44 0.87 0.8  0.51 0.19 0.96 0.15]
 [0.   0.57 0.27 0.21 0.81 0.79 0.11 0.22 0.71 0.74]
 [0.71 0.05 0.38 0.34 0.84 0.87 0.52 0.97 0.38 0.09]
 [0.31 0.51 0.29 0.71 0.55 0.49 0.57 0.77 0.63 0.4 ]
 [0.45 0.53 0.48 0.97 0.64 0.99 0.17 0.21 0.52 0.01]
 [0.88 0.79 0.11 0.48 0.04 0.97 0.66 0.32 0.49 0.64]
 [0.33 0.23 0.61 0.39 0.38 0.95 1.   0.48 0.76 0.03]
 [0.75 0.23 0.55 0.63 0.11 0.1  0.82 0.39 0.68 0.41]
 [0.38 0.51 0.71 0.75 0.04 0.5  0.57 0.78 0.4

In [283]:
for i in range(100):
    print ('eigenvector%s' %i)

eigenvector0
eigenvector1
eigenvector2
eigenvector3
eigenvector4
eigenvector5
eigenvector6
eigenvector7
eigenvector8
eigenvector9
eigenvector10
eigenvector11
eigenvector12
eigenvector13
eigenvector14
eigenvector15
eigenvector16
eigenvector17
eigenvector18
eigenvector19
eigenvector20
eigenvector21
eigenvector22
eigenvector23
eigenvector24
eigenvector25
eigenvector26
eigenvector27
eigenvector28
eigenvector29
eigenvector30
eigenvector31
eigenvector32
eigenvector33
eigenvector34
eigenvector35
eigenvector36
eigenvector37
eigenvector38
eigenvector39
eigenvector40
eigenvector41
eigenvector42
eigenvector43
eigenvector44
eigenvector45
eigenvector46
eigenvector47
eigenvector48
eigenvector49
eigenvector50
eigenvector51
eigenvector52
eigenvector53
eigenvector54
eigenvector55
eigenvector56
eigenvector57
eigenvector58
eigenvector59
eigenvector60
eigenvector61
eigenvector62
eigenvector63
eigenvector64
eigenvector65
eigenvector66
eigenvector67
eigenvector68
eigenvector69
eigenvector70
eigenvector71
ei